## Notebook LM with Amazon Bedrock Claude Sonnet
This notebook is inspired by Audio overview feature of Google Notebook LM. It generates podcast style audio discussions of uploaded PDF document. 

You can upload any PDF document, extracts the content organizes them into interesting themes and generates questions and answers between podcast host and a guest (usually the document author). 

Below is the solution architecture. 

<img src="images/aws-notebooklm-architecture.png" width=500>

You can upload any PDF file. Data is extratced with pypdf library, cleaned-up with custom regex expressions. This is sent to Amazon Bedrock Claude 3.0 Sonnet model with a custom prompt that generates a podcast transcript. The transcript is split as segments. These audio segments are passed to Amazon Polly, a text-to-speech service, to generate the audio clips for each segment of the discussion. Finally with pydub, segments are combined as a single audio file.  

Amazon Polly enables existing applications to speak as a first class feature and creates the opportunity for entirely new categories of speech-enabled products, from mobile apps and cars, to devices and appliances. Amazon Polly includes dozens of lifelike voices and support for multiple languages, so you can select the ideal voice and distribute your speech-enabled applications in many geographies.

### Install dependecies

In [14]:
!pip install boto3 --upgrade

In [ ]:
!pip install pypdf --upgrade

In [ ]:
!pip install pydub --upgrade

### Install ffmpeg & check
sudo apt-get update
sudo apt-get install ffmpeg

ffmpeg -version

### Restart Kernel after the installs

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

In [1]:
%load_ext autoreload
%autoreload 2
import utils
from utils import *

### Process PDF, generate transcript and audio

In [2]:
import boto3

In [3]:
set_audio_dir("audio")

In [4]:
text = pdf_to_text("pdfs/sample.pdf")

In [5]:
system_prompt = get_system_prompt(host_name)

In [7]:
guest_name,title,extracted_speech = generate_podcast_script(system_prompt,text)

In [8]:
guest_name, title

('Ashish Vaswani',
 'Exploring the Transformer: A Revolutionary Attention-Based Model')

In [9]:
extracted_speech

[{'speaker': 'Amy',
  'speech': '<speak>OK listeners, today we have a fascinating discussion on the Transformer, a groundbreaking neural network architecture that has revolutionized sequence modeling tasks like machine translation. Joining me is Ashish Vaswani, one of the lead researchers behind this innovative model from Google Brain. Welcome Ashish!</speak>'},
 {'speaker': 'Ashish Vaswani',
  'speech': "<speak>Thanks for having me Amy. I'm excited to share insights about the Transformer and how it departs from traditional recurrent and convolutional models by relying entirely on an attention mechanism.</speak>"},
 {'speaker': 'Amy',
  'speech': "<speak>Absolutely, let's dive right in. The Transformer replaces the recurrent layers commonly used in sequence models with self-attention layers. Can you explain what self-attention is and how it works in this context?</speak>"},
 {'speaker': 'Ashish Vaswani',
  'speech': "<speak>Certainly. <break time='1s'/> Self-attention is a mechanism th

In [10]:
transcript = '\n'.join(af['speech'] for af in extracted_speech)

In [11]:
sfiles = generate_audio_files(guest_name,extracted_speech)

In [12]:
podcast_file = combine_files(title,sfiles)

In [13]:
podcast_file

'audio/Exploring the Transformer: A Revolutionary Attention-Based Model.mp3'